# Gerando informações para a apresentação

Este notebook não possui nenhuma informação especial.
Serve apenas de auxílio na geração das imagens utilizadas para animação.

In [1]:
# python standard
import os
import time
import re
from multiprocessing import Pool, TimeoutError
from datetime import datetime

# third-party
import numpy as np
import pandas as pd
import scipy.constants as cte
from scipy.integrate import simps
from scipy.sparse import diags
from scipy.linalg import inv
from scipy.fftpack import fft, ifft, fftfreq
from scipy.stats import norm, skewnorm
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
from scipy.special import legendre, expit
from scipy.signal import gaussian

# locals
from core.onda_plana import evolucao_analitica, evolucao_numerica
from core.oscilador_harmonico import solucao_analitica, solucao_numerica
from core.utilidades import *

In [2]:
import locale
locale.setlocale(locale.LC_NUMERIC, "pt_BR.UTF-8")

import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('mestrado')

In [3]:
# ## Constantes físicas
au_l = cte.value('atomic unit of length')
au_t = cte.value('atomic unit of time')
au_e = cte.value('atomic unit of energy')
au_v = cte.value('atomic unit of electric potential')
au_ef = cte.value('atomic unit of electric field')
me = cte.value('electron mass')
c = cte.value('speed of light in vacuum')
q = cte.value('elementary charge')
hbar_ev = cte.value('Planck constant over 2 pi in eV s')
hbar = cte.value('Planck constant over 2 pi')
h = cte.value('Planck constant')
ev = cte.value('electron volt')
# outras relacoes de interesse
au2ang = au_l / 1e-10
au2ev = au_e / ev
hbar_au = 1.0
me_au = 1.0

# onda plana
grid_tempo = np.linspace(0.0, 2.1739442773545673e-14, 20)
alias = {
    'Solucao Analitica': '',
    'Pseudo-Espectral': 'pe',
    'Crank-Nicolson': 'cn',
    'Runge-Kutta': 'rk'
}
metodos = list(alias.keys())


parametros = {
    'onda_plana_parametro_bom': {
        'L': 100.0,
        'N': 1024,
        'dt': 1e-18
    },
    'onda_plana_parametro_ruim': {
        'L': 850.0,
        'N': 256,
        'dt': 1e-16
    }
}

# Gerando onda plana

In [ ]:
# for par in list(parametros.keys()):
#     data = []
#     L = parametros[par]['L']
#     N = parametros[par]['N']
#     dt = parametros[par]['dt']

#     for tempo in grid_tempo:
#         for metodo in metodos:
#             if metodo == 'Solucao Analitica':
#                 evolucao = evolucao_analitica(L=L, tempo=tempo, N=2048)
#                 zf = evolucao['zf_real']
#             else:
#                 evolucao = evolucao_numerica(L=L, N=N, dt=dt, zf=zf,
#                                              method=alias[metodo])
#             z_ang = evolucao['z_ang']
#             psi2 = (np.abs(evolucao['wave_final']) ** 2).real
#             for z,f in zip(z_ang, psi2):
#                 data.append({
#                     'z': z,
#                     'psi2': f,
#                     'metodo': metodo,
#                     'tempo': "{:.2e}".format(tempo)
#                 })
#     dataset = pd.DataFrame(data)
#     dataset.to_csv('saidas/{}.csv'.format(par), sep=',', index=False)
#     dataset.to_pickle('saidas/{}.pkl'.format(par))

In [ ]:
dataset = pd.read_pickle('saidas/onda_plana_parametro_bom.pkl')
for i, t in enumerate(grid_tempo):
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$z$ (\AA)')
    ax.set_ylabel(r'$|\psi(z,t)|^2$')
    tempo = "{:.2e}".format(t)
    for metodo in metodos:
        dataset_by_year = dataset[dataset['tempo'] == tempo]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['metodo'] == metodo]
        x = list(dataset_by_year_and_cont['z'])
        y = list(dataset_by_year_and_cont['psi2'])
        ax.plot(x, y, label=metodo)
    ax.set_title(r"$t = {}$ s".format(as_si(t, 2)), fontsize=16)
    plt.ylim(-0.001, 0.05)
    plt.xlim(-50.0, 50.0)
    plt.legend()
    plt.savefig('apresentacao/saidas/onda_plana_parametros_bons_{0:02d}.png'.format(i))
    plt.show()

In [ ]:
dataset = pd.read_pickle('saidas/onda_plana_parametro_ruim.pkl')
for i, t in enumerate(grid_tempo):
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$z$ (\AA)')
    ax.set_ylabel(r'$|\psi(z,t)|^2$')
    tempo = "{:.2e}".format(t)
    for metodo in metodos:
        dataset_by_year = dataset[dataset['tempo'] == tempo]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['metodo'] == metodo]
        x = list(dataset_by_year_and_cont['z'])
        y = list(dataset_by_year_and_cont['psi2'])
        ax.plot(x, y, label=metodo)
    ax.set_title(r"$t = {}$ s".format(as_si(t, 2)), fontsize=16)
    plt.ylim(-0.001, 0.05)
    plt.xlim(-50.0, 50.0)
    plt.legend()
    plt.savefig('apresentacao/saidas/onda_plana_parametros_ruins_{0:02d}.png'.format(i))
    plt.show()

In [ ]:
dataset = pd.read_pickle('saidas/onda_plana_parametro_ruim.pkl')
for i, t in enumerate(grid_tempo):
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$z$ (\AA)')
    ax.set_ylabel(r'$|\psi(z,t)|^2$')
    tempo = "{:.2e}".format(t)
    for metodo in ['Solucao Analitica', 'Pseudo-Espectral']:
        dataset_by_year = dataset[dataset['tempo'] == tempo]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['metodo'] == metodo]
        x = list(dataset_by_year_and_cont['z'])
        y = list(dataset_by_year_and_cont['psi2'])
        ax.plot(x, y, label=metodo)
    ax.set_title(r"$t = {}$ s".format(as_si(t, 2)), fontsize=16)
    plt.ylim(-0.001, 0.05)
    plt.xlim(-50.0, 50.0)
    plt.legend()
    plt.savefig('apresentacao/saidas/onda_plana_parametros_ruins_stdev_{0:02d}.png'.format(i))
    plt.show()

In [ ]:
dataset = pd.read_pickle('saidas/onda_plana_parametro_ruim.pkl')
for i, t in enumerate(grid_tempo):
    fig, ax = plt.subplots()
    ax.set_xlabel(r'$z$ (\AA)')
    ax.set_ylabel(r'$|\psi(z,t)|^2$')
    tempo = "{:.2e}".format(t)
    for metodo in ['Solucao Analitica', 'Crank-Nicolson', 'Runge-Kutta']:
        dataset_by_year = dataset[dataset['tempo'] == tempo]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['metodo'] == metodo]
        x = list(dataset_by_year_and_cont['z'])
        y = list(dataset_by_year_and_cont['psi2'])
        ax.plot(x, y, label=metodo)
    ax.set_title(r"$t = {}$ s".format(as_si(t, 2)), fontsize=16)
    plt.ylim(-0.001, 0.05)
    plt.xlim(-50.0, 50.0)
    plt.legend()
    plt.savefig('apresentacao/saidas/onda_plana_parametros_ruins_skew_{0:02d}.png'.format(i))
    plt.show()

# Gerando oscilador harmonico

In [ ]:
res = solucao_numerica(iterations=100000, salvar=True)
np.save("saidas/oscilador_harmonico_res_100.npy", res)

In [ ]:
res=np.load("saidas/oscilador_harmonico_res_100.npy")
res = res.item()
V = res['v_ev']
z = res['z_au'] * au2ang
y0 = np.zeros(1024)

dataset=pd.read_pickle('saidas/oscilador_harmonico_100000.pkl')
dataset.iteracoes = dataset.iteracoes+dataset.nivel*100000
dataset.autovalor = dataset.autovalor*au2ev

iteracoes = dataset.iteracoes.unique()
niveis = dataset.nivel.unique()

In [ ]:
for i, iteracao in enumerate(iteracoes):
    fig, ax = plt.subplots()
    ax.plot(z, V, label=r"$V(z)$") 
    for nivel in niveis:
        _iteracao = min((nivel+1)*100000, iteracao)
        if _iteracao <= nivel * 100000:
            continue
        else:
            dataset_by_iter = dataset[dataset['iteracoes'] == _iteracao]
            dataset_by_iter_and_nivel = dataset_by_iter[dataset_by_iter['nivel'] == nivel]
            ev = dataset_by_iter_and_nivel['autovalor'].values[0]
            ev *= float(_iteracao - nivel * 100000)/100000.0
            psi = np.array(dataset_by_iter_and_nivel['autovetor2'])
            psi /= np.sqrt(simps(psi*psi.conj(), z))
            psi = 0.3*psi+ev
            x = dataset_by_iter_and_nivel['z'] * au2ang
            y = psi
            #label = r"$|\phi_{}(z)|^2$".format(nivel)
        ax.plot(x, y)#, label=label)
        
        evlabel = '{:.6f}'.format(ev)
        evlabel = r'$E_{} = {}$ eV'.format(nivel, evlabel.replace('.', ','))
        ax.text(-55, ev+0.02, evlabel)
        ax.text(30, ev+0.02, r'$|\varphi_{} (z)|^2$'.format(nivel))
       
    ax.set_title(r"Itera\c{c}\~{a}o = " + r"{}".format(iteracao), fontsize=16)
    plt.xlim(-60.0, 60.0)
    plt.ylim(-0.1, 1.0)
    plt.legend()
    plt.savefig('apresentacao/saidas/oscilador_harmonico_evoluindo_{0:03d}.png'.format(i))
    plt.show()